In [1]:
import pandas as pd
import io
import string
import numpy as np

In [2]:
# stringified table
data_string = ""'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'""

In [3]:
data = io.StringIO(data_string)

In [4]:
# import data
raw_df = pd.read_csv(data, sep=";")

In [5]:
raw_df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055.0,london_MONTreal


In [6]:
# Capitalize all the letters to improve readability
raw_df['To_From'] = raw_df['To_From'].str.upper()

In [7]:
# Split the To_From column on '_' into two columns
raw_df['To'], raw_df['From'] = raw_df['To_From'].str.split('_', 1).str

In [8]:
# Remove all the punctations and digits
raw_df['Airline Code'] = raw_df['Airline Code'].str.replace(r'[^a-zA-Z ]+', '')

In [9]:
def modify_flight(df):
    '''
    This function is to help modify the FlightCodes column by changing the value type to 
    integer and deal with the missing value where flight codes are supposed to increase by 10 with each row. 
    It will return a new dataframe with modified FlightCodes column. 
    
    Args: df is a dataframe
    
    Return: dataframe
    '''
    idx = 0
    firstNotNull = 0
    isNull = True
    while idx < len(df.index):
        if isNull:
            if (pd.isna(df.iloc[idx,2])):
                firstNotNull += 1
            elif (firstNotNull != 0):
                isNull = False
                for i in reversed(range(firstNotNull)):
                    df.iloc[i,2] = df.iloc[i+1,2] - 10
            elif (firstNotNull == 0):
                isNull = False
                idx += 1
                continue
        else:
            df.iloc[idx,2] = df.iloc[idx-1,2] + 10
        idx += 1
    return df      

In [10]:
modify_flight(raw_df)

,Airline Code,DelayTimes,FlightCodes,To_From,To,From
0,Air Canada,"[21, 40]",20015.0,WATERLOO_NEWYORK,WATERLOO,NEWYORK
1,Air France,[],20025.0,MONTREAL_TORONTO,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035.0,CALGARY_OTTAWA,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045.0,OTTAWA_VANCOUVER,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055.0,LONDON_MONTREAL,LONDON,MONTREAL


In [12]:
# Convert FlightCodes column to integer type
raw_df['FlightCodes'] = raw_df['FlightCodes'].astype(int)

In [14]:
df = raw_df[['Airline Code', 'DelayTimes', 'FlightCodes', 'To', 'From']]

In [15]:
# Desired table output
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL
